<a href="https://colab.research.google.com/github/Nik8x/Deep_writing_generating_text/blob/master/deep_writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import regex as re

import nltk
from nltk.draw.dispersion import dispersion_plot
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
nltk.download('averaged_perceptron_tagger')
from nltk.util import ngrams
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

from sklearn.preprocessing import LabelBinarizer

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

import urllib

import os

# Any results you write to the current directory are saved as output.
# from google.colab import drive
# drive.mount('/content/gdrive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
# print("The current directory is: ", os.getcwd())
# import os
# os.chdir("/content/gdrive/My Drive/Galvanize Adm/Marcel Proust")
# print("The current directory is: ", os.getcwd())

In [0]:
txt = urllib.request.urlopen('http://www.textfiles.com/stories/3gables.txt').read().decode('utf8')
txt = txt.replace('\n', ' ')[322:]
txt[0:400]

"The Adveniure of the Three Gables\r \r   I don't think that any of my adventures with Mr. Sherlock\r Holmes opened quite so abruptly, or so dramatically, as that\r which I associate with The Three Gables. I had not seen Holmes\r for some days and had no idea of the new channel into which his\r activities had been directed. He was in a chatty mood that\r morning, however, and had just settled me into the "

In [0]:
# Let's do some cleaning

In [0]:
words = txt.split(' ')
words_s = [word.lower() for word in words if word.isalpha()]

In [0]:
len(words_s)

4448

In [0]:
# |# creating characters, words and lists 
# characters = sorted(list(set(txt)))
# words = txt.split(' ')
# sentences = txt.split('.')

In [0]:
# # let's remove stop words from words
# stop_words = set(stopwords.words('english'))
# words_s = [i for i in words if not i in stop_words]
# words_s = [word.lower() for word in words_s if word.isalpha()]

In [0]:
# fdist = FreqDist(words_s) # checking most frequent words in whole document
# plt.figure(figsize= (12,2))
# plt.bar(pd.DataFrame(fdist.most_common()[0:20])[0], pd.DataFrame(fdist.most_common()[0:20])[1])
# plt.xticks(rotation=90)
# plt.show()

In [0]:
# # lemmatizing
# lemmatizer = WordNetLemmatizer()
# words_s_l = pd.Series(words_s).apply(lambda x: lemmatizer.lemmatize(x))

In [0]:
# words_s_l_p = pd.DataFrame(nltk.pos_tag(words_s_l), columns = ['words', 'pos'])

In [0]:
sentences = ' '.join(words_s)

In [0]:
# Let's look at overall sentiment of the book
scores = SentimentIntensityAnalyzer().polarity_scores(sentences)
print(scores)

{'neg': 0.074, 'neu': 0.801, 'pos': 0.125, 'compound': 0.9998}


In [0]:
# saving sentences as bigrams

In [0]:
def n_gram(x):
  tokens = [token for token in x.split(" ") if token != ""]
  output = list(ngrams(tokens, 3))
  return output

In [0]:
df = pd.DataFrame(pd.Series(sentences).apply(lambda x: n_gram(x))[0])

# make a dataframe with 2 words as independent and just next third word as target column
df = pd.concat([df[0] + ' ' + df[1], df[2]], axis = 1)
df.head()

,0,2
0,the adveniure,of
1,adveniure of,the
2,of the,three
3,the three,i
4,three i,think


In [0]:
df.shape

(4446, 2)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
X = tf.fit_transform(df[0]).toarray()

In [0]:
X.shape

(4446, 1046)

In [0]:
encoder = LabelBinarizer()
y = encoder.fit_transform(df[2])
y.shape

(4446, 1048)

In [0]:
# LSTMs accept input in the form of (number_of_sequences, length_of_sequence, number_of_features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [0]:
model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 1046, 100)         40800     
_________________________________________________________________
dropout_9 (Dropout)          (None, 1046, 100)         0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1048)              105848    
Total params: 227,048
Trainable params: 227,048
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X, y, epochs = 1, batch_size=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
4446/4446 [==============================] - 400s 90ms/step - loss: 6.3253


In [0]:
y_pred = model.predict(X)

In [0]:
(encoder.inverse_transform(y_pred)).tolist()

['the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
